In [52]:
import os
import pandas as pd
import optuna
from typing import Tuple
import matplotlib.pyplot as plt
import seaborn as sns
os.chdir('/Users/user/PycharmProjects/Shell_hachakton_2023')
from src.commons.utils import get_depot_param_space, get_refinery_param_space
from src.supply_chain_opt.greedy_algorithm import Greedy_algorithm
import warnings
warnings.filterwarnings('ignore')

In [53]:
data = pd.read_csv('data/Biomass_History.csv', index_col=0)
dist = pd.read_csv('data/Distance_Matrix.csv', index_col=0)
biomass_2010 = data['2010']

# Greedy Algorithm
GA = Greedy_algorithm(biomass = biomass_2010,
                      dist = dist)

# Get space for depot optimization
space_depot, number_of_depots = get_depot_param_space(biomass=biomass_2010)
GA.number_of_depots = number_of_depots

Optimal number of depots is 18.


In [54]:
def optuna_obj(trial):

    # Generate space
    space = [trial.suggest_int('depot_idx_{}'.format(i), 0, len(GA.biomass) - 1) for i in range((GA.number_of_depots))]
    space.append(trial.suggest_float('percentage_unilized', 0.5, 1))

    # Constraints
    c1 = GA.depot_constraint(space)
    c2 = GA.harvest_constraint(space)
    trial.set_user_attr('constraint', (c1,c2))

    GA.optimize = False
    cost, _, _ = GA.objective_depot(space)

    return cost

In [55]:
def constraint_func(trial: optuna.Trial) -> Tuple[float]:
    return trial.user_attrs['constraint']

sampler = optuna.samplers.TPESampler(constraints_func=constraint_func)
study = optuna.create_study(sampler=sampler, direction='minimize')  # Create a new study.
study.optimize(optuna_obj, n_trials=100)

[I 2023-08-07 18:22:44,233] A new study created in memory with name: no-name-c83ea7d9-29f0-4713-a264-f560578f5e08
[I 2023-08-07 18:22:44,787] Trial 0 finished with value: 297188.85849794146 and parameters: {'depot_idx_0': 2332, 'depot_idx_1': 1846, 'depot_idx_2': 604, 'depot_idx_3': 601, 'depot_idx_4': 1579, 'depot_idx_5': 1323, 'depot_idx_6': 1619, 'depot_idx_7': 1903, 'depot_idx_8': 1121, 'depot_idx_9': 1331, 'depot_idx_10': 2266, 'depot_idx_11': 1952, 'depot_idx_12': 387, 'depot_idx_13': 476, 'depot_idx_14': 260, 'depot_idx_15': 398, 'depot_idx_16': 1158, 'depot_idx_17': 1060, 'percentage_unilized': 0.5056434508972291}. Best is trial 0 with value: 297188.85849794146.
[I 2023-08-07 18:22:45,247] Trial 1 finished with value: 290019.9301442778 and parameters: {'depot_idx_0': 2002, 'depot_idx_1': 1441, 'depot_idx_2': 2173, 'depot_idx_3': 1387, 'depot_idx_4': 1950, 'depot_idx_5': 1306, 'depot_idx_6': 369, 'depot_idx_7': 1046, 'depot_idx_8': 663, 'depot_idx_9': 656, 'depot_idx_10': 1266, 

In [56]:
optuna.visualization.plot_optimization_history(study)

In [57]:
params = list(study.best_params.values())
params

[2279,
 681,
 2300,
 938,
 504,
 725,
 990,
 925,
 2167,
 676,
 1008,
 1,
 2015,
 1491,
 2247,
 2015,
 2278,
 1333,
 0.9743755970211792]

In [58]:
GA.optimize = False
cost, supply, solution = GA.objective_depot(params)
cost

37373.73978040606

In [59]:
# Depot constraint is violated?
(supply > 20e3).any()

True

In [60]:
# Harvest constraint is violated?
supply.sum() < 0.8 * biomass_2010.sum()

False

In [ ]:
year = '2010'

plt.figure(figsize=(10, 8))
ax = sns.scatterplot(data=data, x='Longitude', y='Latitude', hue=year, palette='viridis')

norm = plt.Normalize(data[year].min(), data[year].max())
sm = plt.cm.ScalarMappable(cmap="viridis", norm=norm)
ax.get_legend().remove()
ax.figure.colorbar(ax=ax, mappable=sm)

for loc in res_depot.x[:-1]:
    y, x = data.iloc[loc, [0]].values[0], data.iloc[loc, [1]].values[0]
    ax.scatter(x, y, label=f'Depot {loc} location', marker='*', color='black', linewidth=5)
#for loc in res_refinery.x:
    #y, x = data.iloc[loc, [0]].values[0], data.iloc[loc, [1]].values[0]
    #ax.scatter(x, y, label=f'Refinery {loc} location', marker='s', color='red', linewidth=5)

plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Spatial Biomass Distribution')